In [1]:
# !pip install geopy

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pprint
import geopandas as gpd
import json
from shapely.geometry import shape, Point  
from geopandas import GeoDataFrame
from shapely.geometry import box
import random
import folium
from matplotlib.colors import to_hex
from keplergl import KeplerGl
import numpy as np
from sklearn.neighbors import BallTree


In [3]:
df = pd.read_csv("C:/AIDC/PythonProject/lec01/gangnamgu_final.csv")

In [4]:

# 1. CSV 파일 불러오기
df_buildings = pd.read_csv("C:/AIDC/PythonProject/lec01/gangnamgu_final.csv")  # 건물 위경도 포함 파일
df_flood = pd.read_csv("C:/AIDC/PythonProject/lec01/강남구침수__이게진짜.csv")  # 실제 침수 위경도 데이터

# 2. 침수 포인트 중 in_gangnam == 1인 애들만 추출
flood_coords = df_flood[df_flood['in_gangnam'] == 1][['lat', 'lon']].dropna().values

# 3. 위도/경도를 라디안(radian) 단위로 변환
build_coords_rad = np.radians(df_buildings[['위도', '경도']].dropna().values)
flood_coords_rad = np.radians(flood_coords)

# 4. BallTree 객체 생성 (haversine 거리 계산 사용)
tree = BallTree(flood_coords_rad, metric='haversine')

# 5. 1km 이내를 라디안 단위로 변환 (지구 반지름 약 6371km)
radius = 1 / 6371

# 6. query_radius로 인덱스 배열 가져오기 (거리 조건 만족하는 포인트들)
results = tree.query_radius(build_coords_rad, r=radius)

# 7. 결과 정리 (침수 여부와 개수)
flood_point_count = [len(r) for r in results]
in_flood_area = [1 if count > 0 else 0 for count in flood_point_count]

# 8. 원본 건물 데이터에 붙이기
df_result = df_buildings.copy()
df_result['in_flood_area'] = in_flood_area
df_result['flood_point_count'] = flood_point_count

# 9. 저장 (선택)
df_result.to_csv("강남구_침수태깅_결과.csv", index=False)

# 10. 미리보기
df_result.columns

Index(['대지위치', '대지면적(㎡)', '건축면적(㎡)', '건폐율(%)', '연면적(㎡)', '용적률(%)', '구조',
       '세대수(세대)', '가구수(가구)', '높이(m)', '지상층수', '지하층수', '승용승강기수', '비상용승강기수',
       '부속건축물수', '부속건축물면적(㎡)', '총동연면적(㎡)', '노후화', '위도', '경도', 'in_flood_area',
       'flood_point_count'],
      dtype='object')

In [5]:
# 필요한 컬럼만 추출
df_flood = pd.read_csv("C:/AIDC/PythonProject/lec01/강남구_침수태깅_결과.csv")

df_flood = df_flood[["대지위치", "지하층수", "노후화", "위도", "경도", "in_flood_area", "flood_point_count", "높이(m)", "지상층수", "구조"]].copy()

# 확인
df_flood.head()

df_flood.to_csv("강남구_침수태깅_결과.csv", index=False)

In [6]:
df["구조"].value_counts()

구조
7    19326
2     3506
6      453
5      443
1      106
0       95
4       64
3       32
Name: count, dtype: int64

In [7]:
# 확인
df_flood

,대지위치,지하층수,노후화,위도,경도,in_flood_area,flood_point_count,높이(m),지상층수,구조
0,서울특별시 강남구 역삼동 628-12번지,2,8,37.503944,127.035095,1,4367,31.44,8,7
1,서울특별시 강남구 역삼동 628-13번지,2,37,37.503733,127.035120,1,4405,25.53,6,7
2,서울특별시 강남구 역삼동 628-14번지,2,37,37.503499,127.035229,1,4492,31.60,8,7
3,서울특별시 강남구 역삼동 628-15번지,1,9,37.504154,127.034695,1,4456,22.75,7,7
4,서울특별시 강남구 역삼동 629번지,1,22,37.503112,127.033016,1,5002,10.95,4,7
...,...,...,...,...,...,...,...,...,...,...
24020,서울특별시 강남구 논현동 218-12번지,1,28,37.512281,127.034913,1,2893,12.55,4,7
24021,서울특별시 강남구 논현동 218-13번지,1,28,37.512244,127.034764,1,2945,12.20,4,7
24022,서울특별시 강남구 논현동 218-14번지,1,28,37.512113,127.034817,1,2950,9.00,4,7
24023,서울특별시 강남구 논현동 218-15번지,0,22,37.512150,127.034964,1,2900,10.78,4,7


In [8]:
# 파일 불러오기
df = pd.read_csv("C:/AIDC/PythonProject/lec01/강남구_침수태깅_결과.csv")

# 1층단층높이 계산
df["1층단층높이추정"] = (df["높이(m)"] / df["지상층수"]) * 1.1
df.loc[df["지상층수"] == 0, "1층단층높이추정"] = None


# 파일 저장 (index 없이)
df.to_csv("C:/AIDC/PythonProject/lec01/강남구_침수태깅_결과.csv", index=False)

df


,대지위치,지하층수,노후화,위도,경도,in_flood_area,flood_point_count,높이(m),지상층수,구조,1층단층높이추정
0,서울특별시 강남구 역삼동 628-12번지,2,8,37.503944,127.035095,1,4367,31.44,8,7,4.32300
1,서울특별시 강남구 역삼동 628-13번지,2,37,37.503733,127.035120,1,4405,25.53,6,7,4.68050
2,서울특별시 강남구 역삼동 628-14번지,2,37,37.503499,127.035229,1,4492,31.60,8,7,4.34500
3,서울특별시 강남구 역삼동 628-15번지,1,9,37.504154,127.034695,1,4456,22.75,7,7,3.57500
4,서울특별시 강남구 역삼동 629번지,1,22,37.503112,127.033016,1,5002,10.95,4,7,3.01125
...,...,...,...,...,...,...,...,...,...,...,...
24020,서울특별시 강남구 논현동 218-12번지,1,28,37.512281,127.034913,1,2893,12.55,4,7,3.45125
24021,서울특별시 강남구 논현동 218-13번지,1,28,37.512244,127.034764,1,2945,12.20,4,7,3.35500
24022,서울특별시 강남구 논현동 218-14번지,1,28,37.512113,127.034817,1,2950,9.00,4,7,2.47500
24023,서울특별시 강남구 논현동 218-15번지,0,22,37.512150,127.034964,1,2900,10.78,4,7,2.96450


In [9]:
# 기존 구조 코드 값을 일괄적으로 직접 변경 (기존 '구조' 컬럼 값 자체를 덮어쓰기)

structure_update_map = {
    7: 6, # 철근콘크리트구조
    6: 5, # 철골철근콘크리트구조
    5: 4, # 일반철골구조
    4: 3, # 일반목구조
    3: 2, # 조적조 통합(블록)
    2: 2 #  조적조 통합(벽돌)
}

# 나머지는 1로 기타 처리 (그 외 혹은 조립식판넬)
df['구조'] = df['구조'].map(structure_update_map).fillna(1).astype(int)


df.to_csv("C:/AIDC/PythonProject/lec01/강남구_침수태깅_결과.csv", index=False)

In [10]:
# 결과 상위 20개 확인
df['구조'].value_counts()

구조
6    19326
2     3538
5      453
4      443
1      201
3       64
Name: count, dtype: int64

In [14]:
df['in_flood_area'].value_counts()  # 0빼고 테스트사이즈 0.2로해서 피해량 데이터

in_flood_area
1    16793
0     7232
Name: count, dtype: int64